In [3]:
!pip install -q plotly tqdm
#import tools
!wget https://raw.githubusercontent.com/avrymi-asraf/collective-motion/main/tools.py

--2023-08-15 19:44:47--  https://raw.githubusercontent.com/avrymi-asraf/collective-motion/main/tools.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8571 (8.4K) [text/plain]
Saving to: ‘tools.py.1’

tools.py.1          100%[===================>]   8.37K  --.-KB/s    in 0s      

2023-08-15 19:44:47 (70.3 MB/s) - ‘tools.py.1’ saved [8571/8571]



In [2]:
from tools import *

In [6]:
# @title Import libraries
import timeit
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff

import pandas as pd
import numpy as np

import tqdm.notebook as tqdm



In [5]:
# @title typing
from typing import Tuple, Callable, Union, List, Optional, Any

Opt = Optional
NumType = Union[int, float,torch.Tensor]
TensorType = torch.Tensor
BoxDimType = Tuple[NumType, NumType]
RangeType = Tuple[NumType, NumType]
LocType = Tuple[NumType, NumType]
IndScalarType = Union[int, TensorType]
UpdateFuncType = Callable[[TensorType, dict], TensorType]
IndexsType = Union[TensorType, List[IndScalarType], Tuple[IndScalarType, ...], range]
OptimazerType = Union[optim.Optimizer, optim.Adam, optim.SGD, optim.RMSprop]
LossType = Union[nn.Module, nn.CrossEntropyLoss, nn.MSELoss, nn.BCELoss]

<div dir="rtl" lang="he" xml:lang="he">

# מודלים טובים ומודלים טובים יותר


כאשר אנחנו חושבים על מודלים בקוגניציה, אנחנו חושבים על מודלים שמסבירים תופעה במונחים של קוגניציה.
לדוגמה מודל
_vicsek_
מסביר את בעלי החיים נעים בלהקה, ע"י מערכת קשרים בן הצורך לשמור על מרחק מהשני, הצורך להתאים את המהירות למהירות של השכנים וכו'.
הבעיה במודלים כאלה היא שהם כמעט לא ניתנים לבדיקה, הרעש גדול מידי.
אם ננסה לבדוק את ה
_loss_
של פונק

תנועה קולקטיבית היא אחד התופעות המענייונות ביותר,
קבוצה של בעלי חיים נעה במרחב בצורה כאילו מאורגנת ללא שוב מנהיג נראה לעין.
אלו מודלים יכולים להסביר את התנועה הזאת?
והאם נמצא דרך להשוות בן המודלים האלו?

<div dir="rtl" lang="he" xml:lang="he">

## הדאטה איתו נעבוד
נשתמש בדאטה של תנועת זגי זברה.
זה יאפשר לנו לבדוק את התנועה במישור

In [7]:
# data is a tensor of shape (t,m,2)
# where t is the number of time steps and m is the number of objects.
# The last dim is (x,y) cordination in the plane.
data = torch.load("dataset/60/1.pt")
# using the cordination we can add for every point the speed, acceleration, and angle.
# so now data is a tensor of shape (t,m,5) where for every point we have (x,y, speed, acceleration,angle)
data = add_parameters(data).nan_to_num(nan=0.0)
#data for first point at first time step
print(data[0,0])
#We assume that the arena is square
print(f'{data[:,:,:2].max()}')
px.histogram(data[0,:,2], title="speed distribution")

FileNotFoundError: ignored

<div dir="rtl" lang="he" xml:lang="he">

כמו שניתן לראות, המיקום נתון בפיקסלים, המהירות היא בטווח של 0-30 והזוית היא ברדיאנים.

ננרמל את המיקום להיות סביב ה0 בטווח של +-5
ואת המהירות להיות בטווח של 0-10

In [ ]:
#default of function  is
data = normelize_data(data,max_loc=3580)

In [ ]:
plot_timeline_with_direction(data,"first look",steps=range(0,100))

<div dir="rtl" lang="he" xml:lang="he">

יש סדר מסויים בתנועה, אבל היא מורעשת מאוד,
ננסה לראות מה מודל תנועה היה נותר למצב כזה.

<div dir="rtl" lang="he" xml:lang="he">

## מודל מורחב
כל בעל חיים מתואר ע"י מיקומו,
מסומן ב
$\vec{r}_i \in R^2$
ווקטור הכיוון שלו שמסומן ב
$\vec{v}_i \in R^1$
כאשר כלל העידכון הוא הליכה בכיון הממוצע.
ע"י חישוב וקטור הכיון של השכנים, מתואר ע"י המשוואה
$$
\\ \vec{S}_i = \sum_{j\in \text{Neg}(i)}{\vec{v}_j}
\\ \vec{v}_i = \frac{\vec{S}_i}{|\vec{S}_i|}*v_0
$$
כאשר
$v_0$
הוא גודל קבוע המייצג את מהירות בעל החיים.




$$
\\ \vec{S} = \sum_{j\in \text{Neg}(i)}{\vec{v}_j}
\\ \vec{v}_i = \frac{\vec{S}}{|\vec{S}|}*v_0
$$

In [ ]:
def model(
    data: TensorType,
    parametes={},
) -> TensorType:
    """
    update the data according to vicsek model

    Args:
        data (TensorType): tensor (N,4) of x,y,speed,angel
        radius (float, optional): radius which one neighbors.Defaults to 500. in pixsels.
        parametes (dict, optional): {radius, update_ratio_angle, update_ratio_speed, update_ratio_acceleratoin}
        of the update for angle, speed, and. Defaults to (500, 0.7, 0.7, 0.7).

    Returns:
        TensorType: tensor (N,4) new data
    """
    dufualt_patameters = {
        "radius": 1,
        "update_ratio_angle": 0.7,
        "update_ratio_speed": 0.7,
        "noise_angle": 1e-2,
        "noise_speed": 1e-3,
    }
    dufualt_patameters.update(parametes)

    radius = dufualt_patameters["radius"]
    update_ratio_angle = dufualt_patameters["update_ratio_angle"]
    update_ratio_speed = dufualt_patameters["update_ratio_speed"]
    noise_angle = dufualt_patameters["noise_angle"]
    noise_speed = dufualt_patameters["noise_speed"]
    # get the neighbors matrix, is matrix (N,N) where each row is the neighbors of the data point
    # so data[neighbors[i]] is the neighbors of data[i]
    neighbors = get_index_neighbors(data[:, :2], radius)
    # get the avrage angle of the neighbors of each data point
    avrage_angle = (
        torch.stack(
            [
                update_ratio_angle * data[neighbors[i], 3].mean(dim=0)
                + (1 - update_ratio_angle) * data[i, 3]
                if neighbors[i].any()
                else data[i, 3]
                for i in range(len(data))
            ]
        )
        + torch.randn(len(data)) * noise_angle
    )
    # get the avrage velocity of the neighbors of each data point
    avrage_speed = (
        torch.stack(
            [
                update_ratio_speed * data[neighbors[i], 2].mean(dim=0)
                + (1 - update_ratio_speed) * data[i, 2]
                if neighbors[i].any()
                else data[i, 2]
                for i in range(len(data))
            ]
        )
        + torch.randn(len(data)) * noise_speed
    )

    # update the coordinates
    new_x = avrage_speed * torch.cos(avrage_angle) + data[:, 0]
    new_y = avrage_speed * torch.sin(avrage_angle) + data[:, 1]
    return torch.column_stack([new_x, new_y, avrage_speed, avrage_angle])

In [ ]:
# create a timeline for data, so data[t] is represent the time t given by f(data[t-1])
ret = create_timeline_series(
    data[:1],
    model,
    100,
    {
        "radius": 0.5,
        "update_ratio_angle": 0.7,
        "update_ratio_speed": 0.7,
        "noise_angle":1e-1
    },
)

In [ ]:
plot_timeline_with_direction(ret, "test").show()

<div dir="rtl" lang="he" xml:lang="he">

ניתן לראות שהמודל הזה לא מספיק, וברור שבטווח הרחוק הוא לא יצליח לחזות את ההתנהגות,
בעיקר שאין לנו בתוך המודל את הזירה, זה שהזירה עגולה משפיע על הסיבוב של הדגים.


במקום לבחון את ההתהגות לטווח רחוק ננסה למצוא את הפרמטרים שחוזים את הצעד הבא בצורה הטובה ביותר.

<div dir="rtl" lang="he" xml:lang="he">

## למידת הפרמטרים של המודל ע"י מחשב
הבעיה בצורה הנוכחית לא ניתנת ללמידה.
(לפחות לפי הידוע לי...)
ננסה ללמוד אותה בכוח גס, כלומר ניתן רשימת פרמטרים ונבדוק מי מהם עם התוצאה הטובה ביותר.

<div dir="rtl" lang="he" xml:lang="he">

### פונקציית הערכה
דבר ראשון צריך להגדיר פונקציית הערכה שתגדיר מתי התוצאה טובה ומתי לא.
נשתמש בפונקציית מרחק ריבועי פשוטה.
זאת פונקציה שרגישה למרחקים (עבור מרחקים קטנים מ1 היא קטנהמהערך הלינארי, ועבור מרחקים גדולים מ1 היא גדולה. במקרה שלנו מכיון שלא נירמנלנו את הערכים כל הערכים גדולים מ1)

In [ ]:
def loss_function(pred: TensorType, target:TensorType) -> TensorType:
    return nn.MSELoss()(pred, target)

<div dir="rtl" lang="he" xml:lang="he">

### פרמטרים לבדיקה
לכל אחד מהפרמטרים הבאים נגדיר ערכים רלוונטיים לבדיקה. כל ערך יופיע בפעם אחת בלבד בכל סעיף.

- המקדם של המהירות
$\alpha$
- המקדם של הזווית
$\beta$
- המקדם של התאוצה
$\gamma$
- הרדיוס של השכנים
$r$

נעשה מכפלה קרטזית של כל הערכים ונבדוק בכל אחד מהם מה הערך הטוב ביותר.


In [ ]:
speed = torch.tensor([0.1, 0.25, 0.5, 0.75, 1.0])
angle = torch.tensor([0.1, 0.25, 0.5, 0.75, 1.0])
radius = torch.tensor([0.1, 0.3, 0.5, 0.7, 0.9, 1.1, 1.3, 1.5, 1.7, 1.9, 2.1])

all_options = torch.cartesian_prod(speed, angle, radius)

<div dir="rtl" lang="he" xml:lang="he">

### גודל דגימה
על מנת להשוות בן המודלים נבדוק כל מודל על חלק מדאטה.

In [ ]:
indexes = torch.randint(0, len(all_options), (20,))
parameters_examples = all_options[indexes]
run_data_for_examples = pd.DataFrame(
    {"ind_paramer": int(), "num_time_point": int(), "loss": float()},
    index=range(len(parameters_examples)*30),
)

In [ ]:
size_examples = range(50, 200, 5)
run_ind = 0
for size in tqdm.tqdm_notebook(size_examples):
    for parameter_ind in range(len(parameters_examples)):
        indexes = torch.randint(0, data.shape[0] - 1, (10,))
        parameter = parameters_examples[parameter_ind]
        parameters = {
            "update_ratio_speed": parameter[0].item(),
            "update_ratio_angle": parameter[1].item(),
            "radius": parameter[2].item(),
        }
        y_pred = evaluat_in_indexes(data, indexes, model, parametes=parameters)
        y_true = data[indexes + 1]
        y_pred = torch.nan_to_num_(y_pred, nan=0.0)
        y_true = torch.nan_to_num_(y_true, nan=0.0)
        loss = loss_function(y_pred, y_true)
        run_data_for_examples.iloc[run_ind] = [parameter_ind, size, loss.item()]
        run_ind += 1

  0%|          | 0/30 [00:00<?, ?it/s]

In [ ]:
px.line(run_data_for_examples, x="num_time_point", y="loss", color="ind_paramer").show()

<div dir="rtl" lang="he" xml:lang="he">

נוכל לשים לב שעבור אותם סט פרמטרים "נמוכים" בטווח בין 0.2 ל0 השונות לא גבוהה, בנוסף היא כמעט לא משתפרת כאשר מגדילים את גודל סט הנתונים. לכן נוכל לבחור סט נתונים בגודל 100.
נראה שהוא יספק לנו אומדן טוב מספיק לבחירת פרמטרים למודל

In [ ]:
# define dataframes to store the results
run_data = pd.DataFrame(
    columns=["speed", "angel", "radius", "loss"],
    index=range(len(all_options)),
    dtype=float,
)

<div dir="rtl" lang="he" xml:lang="he">

נריץ את הקוד על כל מבנה של אפשרויות ונראה מי מהם מביא את התוצאה הטובה ביותר

In [ ]:
batch_size = 100
for ind, optoin in tqdm.tqdm_notebook(enumerate(all_options)):
    # choice list of random indexes to evaluate
    indexes = torch.randint(0, data.shape[0] - 1, (batch_size,))
    parameters = {
        "update_ratio_speed": optoin[0],
        "update_ratio_angle": optoin[1],
        "radius": optoin[2],
    }
    y_pred = evaluat_in_indexes(data, indexes, model, parametes=parameters)
    y_true = data[indexes + 1]
    y_pred = torch.nan_to_num_(y_pred, nan=0.0)
    y_true = torch.nan_to_num_(y_true, nan=0.0)
    loss = loss_function(y_pred, y_true)
    run_data.iloc[ind] = [*parameters.values(), loss.item()]

0it [00:00, ?it/s]

<div dir="rtl" lang="he" xml:lang="he">

נמצא את סט הפרמטרים שמביא את המינימום

In [ ]:
run_data.iloc[run_data["loss"].argmin()]

speed     0.50000
angel     0.10000
radius    1.90000
loss      0.06758
Name: 119, dtype: float64

In [ ]:
px.histogram(run_data,x='loss',labels=["speed", "angel", "radius"]).show()

In [ ]:
px.histogram(run_data[run_data['loss']<0.1],x='loss',labels=["speed", "angel", "radius"]).show()

<div dir="rtl" lang="he" xml:lang="he">

ניתן לראות שיש מספר רב של סטים שמביאים תוצאות טובות. ננסה לראות את ההשפעה של כל פרמטר על התוצאה הסופית.

In [ ]:
limit = 1.0
px.line(
    run_data[run_data['loss']<limit].groupby("speed").mean(), y="loss",range_y=[0,0.5], title="Loss vs Speed"
).show()
px.line(
    run_data[run_data['loss']<limit].groupby("angel").mean(),range_y=[0,0.5],y="loss", title="Loss vs angel"
).show()
px.line(
    run_data[run_data['loss']<limit].groupby("radius").mean(), y="loss",range_y=[0,0.5], title="Loss vs radius"
).show()